In [1]:
import argparse
import os
import pickle
import random
import shutil
import time

import matplotlib
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import sklearn.metrics as metrics
import torch
import torch.nn as nn
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from tensorboardX import SummaryWriter
from torch.autograd import Variable
from torch_geometric.utils import to_networkx

import configs
import gengraph
import models
import utils.featgen as featgen
import utils.graph_utils as graph_utils
import utils.io_utils as io_utils
import utils.math_utils as math_utils
import utils.parser_utils as parser_utils
import utils.train_utils as train_utils
from dataset import GraphDataset, create_dataset

In [2]:
G, labels, name = gengraph.gen_syn1(
    feature_generator=featgen.ConstFeatureGen(np.ones(32, dtype=float))
)
num_classes = max(labels) + 1

feat_dict[0]["feat"]: float32
G.nodes[0]["feat"]: float32


In [3]:
print(G[0])

{1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}, 11: {}, 12: {}, 13: {}, 15: {}, 16: {}, 19: {}, 26: {}, 27: {}, 29: {}, 33: {}, 45: {}, 47: {}, 51: {}, 52: {}, 54: {}, 58: {}, 59: {}, 60: {}, 66: {}, 69: {}, 84: {}, 88: {}, 90: {}, 97: {}, 98: {}, 112: {}, 114: {}, 116: {}, 123: {}, 147: {}, 149: {}, 150: {}, 156: {}, 160: {}, 166: {}, 167: {}, 168: {}, 195: {}, 202: {}, 203: {}, 221: {}, 222: {}, 223: {}, 231: {}, 236: {}, 242: {}, 258: {}, 264: {}, 290: {}, 300: {}}


In [4]:
# print(G[0])
# print((labels))

pyg_dataset = pyg_dataset = GraphDataset(
    torch.load("my_data/cycle_line_star_complete_1.pt", weights_only=False)
)

print(pyg_dataset[0])

for data in pyg_dataset:
    G = to_networkx(data, to_undirected=True)

    # Skip graphs without edges
    if G.number_of_edges() == 0:
        print(f"Graph {data} has no edges, skipping.")
        continue

Data(x=[9, 4], edge_index=[2, 58], y=[1], num_nodes=9, node_classes=[9])


In [10]:
graphs = []
all_labels = set()
for data in pyg_dataset:
    G = to_networkx(data, to_undirected=True)

    # Add graph label
    label = data.y.item()
    G.graph["label"] = label
    all_labels.add(label)

    # Add node features
    for i, node in enumerate(G.nodes()):
        G.nodes[node]["feat"] = data.x[i].numpy()

    graphs.append(G)

print(graphs[0])

for i, G in enumerate(graphs):
    print(f"Graph {i} (label: {G.graph['label']}): {G.number_of_edges()} edges")
    if G.number_of_edges() == 0:
        print(f"Adjacency matrix for graph {i}:\n", nx.adjacency_matrix(G).todense())

Graph with 9 nodes and 29 edges
Graph 0 (label: 1): 29 edges
Graph 1 (label: 1): 30 edges
Graph 2 (label: 1): 31 edges
Graph 3 (label: 1): 30 edges
Graph 4 (label: 1): 32 edges
Graph 5 (label: 1): 34 edges
Graph 6 (label: 1): 31 edges
Graph 7 (label: 1): 34 edges
Graph 8 (label: 1): 37 edges
Graph 9 (label: 1): 29 edges
Graph 10 (label: 1): 30 edges
Graph 11 (label: 1): 31 edges
Graph 12 (label: 1): 30 edges
Graph 13 (label: 1): 32 edges
Graph 14 (label: 1): 34 edges
Graph 15 (label: 1): 31 edges
Graph 16 (label: 1): 34 edges
Graph 17 (label: 1): 37 edges
Graph 18 (label: 1): 29 edges
Graph 19 (label: 1): 30 edges
Graph 20 (label: 1): 31 edges
Graph 21 (label: 1): 30 edges
Graph 22 (label: 1): 32 edges
Graph 23 (label: 1): 34 edges
Graph 24 (label: 1): 31 edges
Graph 25 (label: 1): 34 edges
Graph 26 (label: 1): 37 edges
Graph 27 (label: 1): 29 edges
Graph 28 (label: 1): 30 edges
Graph 29 (label: 1): 31 edges
Graph 30 (label: 1): 30 edges
Graph 31 (label: 1): 32 edges
Graph 32 (label: 1